In [1]:
#package imports
from __future__ import division

import numpy as np
import datetime
import os
from wrf import to_np, getvar, get_basemap, latlon_coords
from netCDF4 import Dataset
import pandas as pd
from datetime import timedelta
from itertools import product
import math
import xarray as xr
from numpy.lib.stride_tricks import as_strided
import multiprocessing as mp
from itertools import product

#this script generates error files for perturbed sst runs with times

In [9]:
def run_uh_perturbssts(num, UH_threshold, wrf_file_folder_choice):
    
    file2_wrf_d02 = '/glade/scratch/molina/'+wrf_file_folder_choice+'/wrfout_d01_2017-01-15_09:00:00'
    
    print('basemap file opened...')
    
    ncfile2_wrf_d02 = Dataset(file2_wrf_d02)
    while True:
        dbz2_wrf_d02 = np.array([0])
        if len(dbz2_wrf_d02) == 1:
            try: 
                dbz2_wrf_d02 = getvar(ncfile2_wrf_d02, "dbz")
            except ValueError:
                continue
        if len(dbz2_wrf_d02) != 1:
            print('File Opened...')
            break
    lats4, lons4 = latlon_coords(dbz2_wrf_d02)
    m = get_basemap(dbz2_wrf_d02)
    xMax, yMax = m(m.urcrnrlon, m.urcrnrlat) 
    xMin, yMin = m(m.llcrnrlon, m.llcrnrlat) 
    x_range = (xMax-xMin) / 1000 
    y_range = (yMax-yMin) / 1000 
    
    numXGrids = round(x_range / choice_res + .5,0) 
    numYGrids = round(y_range / choice_res + .5,0)
    xi = np.linspace(xMin, xMax, int(numXGrids))
    yi = np.linspace(yMin, yMax, int(numYGrids))
    
    
    #########################################################################################
    #########################################################################################
    #########################################################################################


    #create the range of dates to loop thru wrf files
    begdate = datetime.datetime.strptime('2017012109',"%Y%m%d%H") 
    enddate = datetime.datetime.strptime('2017012309',"%Y%m%d%H") 
    monthstr = datetime.datetime.strftime(begdate,"%B")
    #format into strings
    bmonth = '%02d' % begdate.month
    bday = '%02d' % begdate.day
    bbday = begdate.day
    #loop thru hours
    dates = []
    while begdate < enddate:
        dates.append(begdate)
        begdate+=datetime.timedelta(hours=1) 

    uvv_coarse = np.zeros((len(dates),yi.shape[0]-1, xi.shape[0]-1))

    counter = 0
    

    for count, dt in enumerate(dates):

        YY = dt.year

        MM = "%02d"%dt.month
        DD = "%02d"%dt.day
        HH = "%02d"%dt.hour

        HH_current = dt.hour

        if count != 0:
            if HH_current != HH_previous:    
                counter += 1

        wrffile = '/glade/scratch/molina/'+wrf_file_folder_choice+'/wrfout_d01_2017-'+MM+'-'+DD+'_'+HH+':00:00'

        if os.path.isfile(wrffile)==True:

            ncfile = Dataset(wrffile)

            uvv = getvar(ncfile, "updraft_helicity", meta=False)
            uvv_val = uvv

            uvv2 = xr.open_dataset(wrffile, decode_cf=True).UP_HELI_MAX.values[0,:,:]
            uvv2_val = uvv2


            for i, j in product(range(0,uvv2.shape[0]), range(0,uvv2.shape[1])):

                max_UH = np.nanmax([uvv_val[i,j],uvv2_val[i,j]])

                if max_UH >= UH_threshold:

                    latsnow, lonsnow = to_np(lats4)[i,j], to_np(lons4)[i,j]
                    x_proj,y_proj = m(lonsnow, latsnow)
                    grid, _, _ = np.histogram2d(np.array([y_proj]),np.array([x_proj]), bins=[yi,xi])

                    uvv_coarse[counter,:,:] += grid > 0

        else:

            print("file doesn't exist", os.path.basename(ncfile))

        HH_previous = dt.hour

        print(str(dt)+' complete...')

    print(f'UH ({UH_threshold}) finished.')

    x1, y1 = xi, yi

    x2 = np.zeros(x1.shape)
    y2 = np.zeros(y1.shape)
    x3 = np.zeros(x1.shape)
    y3 = np.zeros(y1.shape)

    for i, j in product(range(len(x1)),range(len(y1))):

        x2[i], y2[j] = m(x1[i], y1[j], inverse=True)

        x2[i] = x2[i]-0.4
        y2[j] = y2[j]-0.4

        x3[i], y3[j] = m(x2[i], y2[j])

    x4, y4 = x3[1:], y3[1:]

    uvv_coarse1 = np.nansum(uvv_coarse, axis=0)


    ###############################################################################
    ###############################################################################
    ###############################################################################


    final_filed = xr.Dataset({'UH':(['y','x'], uvv_coarse1),
                              'UH_dates':(['time','y','x'], uvv_coarse),                              
                              },

                              coords={'lon':(['x'],x4),
                                      'lat':(['y'],y4),
                                      'time':(['time'],pd.to_datetime(dates))},

                              attrs={'File Author':'Maria J. Molina'})

    final_filed.to_netcdf('/glade/scratch/molina/'+wrf_file_folder_choice+'/vert_grid_80_UH_'+str(UH_threshold)+'.nc')

    print('/glade/scratch/molina/'+wrf_file_folder_choice+'/vert_grid_80_UH_'+str(UH_threshold)+'.nc', 'saved')
    
    return(num)

In [8]:
#computation choosing

choice_res = np.array([80])[0]

UH_thresholds = np.array([100,90,80,70,60,50,40
                         ])

wrf_file_folder_choices = np.array([
    '/WRF1/WRF/run',
    '/WRF2/WRF/run',
    '/WRF3/WRF/run',
    '/WRF4/WRF/run',
    '/WRF5/WRF/run',
    '/WRF6/WRF/run',
    '/WRF7/WRF/run',
    '/WRF8/WRF/run',
    '/WRF9/WRF/run',
    '/WRF10/WRF/run',
    '/WRF11/WRF/run',
    '/WRF12/WRF/run'])

In [ ]:
pool = mp.Pool(36)

results = []

def collect_result(result):
    global results
    results.append(result)
    
    
for num, (UH_threshold, wrf_file_folder_choice) in enumerate(product(UH_thresholds, wrf_file_folder_choices)):
    
    results=pool.apply_async(run_uh_perturbssts, args=(num, UH_threshold, wrf_file_folder_choice), callback=collect_result)
    
pool.close()
pool.join()
print("fully completed woohoo!")